# Enigma - CDL Hackathon - Trip Generator





In [1]:
import networkx as nx
from collections import defaultdict
from dimod import BinaryQuadraticModel
from tabu import TabuSampler        
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
import neal
from strawberryfields.apps import plot
import plotly
import numpy as np
import csv
from flightHelper import *

## Testing the Anneal class


In [2]:

gen = Anneal(dataset="DS2b.csv",homebases={ "LCA" : 1 },atypes=[],depday=1)

gen.prepare()

r = gen.solve(useHyb=True)

print( r, " result(s) found")
print (gen.getN(), " flight segments (nodes)")
print(gen.getN()**2+gen.getN()," qubits")

if ( r > 0 ):
    
    gen.print_all()
    
    for M in gen.getAdjMatrices():
        # Adjacency matrix output
        print("Adjacency Matrix:")
        print(M)
else:
    print("No Results")


Minimum Dep 205
Maximum Arr 1490
{'id': 1, 'lab': '102', 'dep': 'LCA', 'arr': 'ATH', 'deptime': 245, 'arrtime': 350, 'depday': 1, 'arrday': 1, 'ft': 105, 'UT1': 0, 'UT2': 0, 'T': 0, 'ci': 0, 'co': 0, 'DepBaseWgt': 1000000, 'ArrBaseWgt': 0}
{'id': 2, 'lab': '103', 'dep': 'ATH', 'arr': 'LCA', 'deptime': 395, 'arrtime': 490, 'depday': 1, 'arrday': 1, 'ft': 95, 'UT1': 0, 'UT2': 0, 'T': 0, 'ci': 0, 'co': 0, 'DepBaseWgt': 0, 'ArrBaseWgt': 1000000}
{'id': 3, 'lab': '104', 'dep': 'LCA', 'arr': 'ATH', 'deptime': 460, 'arrtime': 565, 'depday': 1, 'arrday': 1, 'ft': 105, 'UT1': 0, 'UT2': 0, 'T': 0, 'ci': 0, 'co': 0, 'DepBaseWgt': 1000000, 'ArrBaseWgt': 0}
{'id': 4, 'lab': '105', 'dep': 'ATH', 'arr': 'LCA', 'deptime': 610, 'arrtime': 705, 'depday': 1, 'arrday': 1, 'ft': 95, 'UT1': 0, 'UT2': 0, 'T': 0, 'ci': 0, 'co': 0, 'DepBaseWgt': 0, 'ArrBaseWgt': 1000000}
{'id': 5, 'lab': '106', 'dep': 'LCA', 'arr': 'ATH', 'deptime': 765, 'arrtime': 870, 'depday': 1, 'arrday': 1, 'ft': 105, 'UT1': 0, 'UT2': 0, 

In [3]:
a = 1/0

ZeroDivisionError: division by zero

## Controls


In [ ]:
# Solver selection. Default is Tabu when all set to False
    
useQPU = False
useNeal = False
useHyb = True

# Prepare the problem specific home base constraints

#targetDataSet="DS2b.csv"
targetDataSet="" # Use the default data set from within
HomeBases = { "LCA" : 1 }

# Set up the home base biases 

NotHomeBaseWeight = makeBaseWeight(len(HomeBases)+1) 

# Constraint and objectives switches

cons_1c_on = True  # Enforce 1 node per row
cons_2_on = True   # Enforce node used only once
cons_3_on = True   # At least one Start flag per result
cons_4_on = True   # Only connections allowed by edges
cons_5_on = True   # Return to start location

objective_1_on = True # Establish baseline unallocated time
objective_2_on = True # Replace unallocated time with CI, CO, gap

LagrangeA = makeBaseWeight(len(HomeBases)+1)    # Order of magnitude for constraints -> Should be bigger than the base constraints magnitude
LagrangeB = LagrangeA / 10                       # Order of magnitude for allowed edges
LagrangeC = 1                                    # order of magnitude for return to start location (uses base weights)

In [ ]:
def buildSet1():
    segments=[]
    segments.append( Node(Segment(1, "101", "LCA", "ATH", 600, 695, 1, 1,HomeBases)))
    segments.append( Node(Segment(2, "102", "ATH", "LCA", 755, 845, 1, 1,HomeBases)))
    segments.append( Node(Segment(3, "103", "LCA", "ATH", 875, 970, 1, 1,HomeBases)))
    segments.append( Node(Segment(4, "104", "ATH", "LCA", 1035, 1125, 1, 1,HomeBases)))
    return(segments)

# Load data from files or use the default data set 1

#segments = loadFlts("DS1.csv", ["320"], HomeBases )
if ( targetDataSet != "" ):
    segments = loadFlts(targetDataSet, Atypes=[], depDay = 1, HomeBases=HomeBases )
else:
    segments = buildSet1()

for s in segments:
    print(s.obj.__dict__)

In [ ]:



def buildSet2():
    segments=[]
    segments.append( Node(Segment(1, '2450', 'HOT', 'ELD', 425, 465, 1, 1,HomeBases)))
    segments.append( Node(Segment(6, '2450', 'ELD', 'IAH', 485, 595, 1, 1,HomeBases)))
    segments.append( Node(Segment(11, '2451', 'IAH', 'ELD', 625, 740, 1, 1,HomeBases)))
    segments.append( Node(Segment(16, '2451', 'ELD', 'HOT', 755, 795, 1, 1,HomeBases)))
    segments.append( Node(Segment(21, '2452', 'HOT', 'ELD', 850, 890, 1, 1,HomeBases)))
    segments.append( Node(Segment(26, '2452', 'ELD', 'IAH', 905, 1015, 1, 1,HomeBases)))
    segments.append( Node(Segment(31, '2453', 'IAH', 'ELD', 1050, 1160, 1, 1,HomeBases)))
    segments.append( Node(Segment(36, '2453', 'ELD', 'HOT', 1175, 1215, 1, 1,HomeBases)))
    segments.append( Node(Segment(41, '2454', 'HOT', 'MEM', 810, 890, 1, 1,HomeBases)))
    segments.append( Node(Segment(46, '2455', 'MEM', 'HOT', 750, 830, 1, 1,HomeBases)))
    segments.append( Node(Segment(51, '2456', 'HOT', 'MEM', 1230, 1310, 1, 1,HomeBases)))
    segments.append( Node(Segment(56, '2457', 'MEM', 'HOT', 330, 410, 1, 1,HomeBases)))
    segments.append( Node(Segment(136, '4125', 'MEM', 'HRO', 920, 1005, 1, 1,HomeBases)))
    segments.append( Node(Segment(142, '4126', 'HRO', 'MEM', 1030, 1110, 1, 1,HomeBases)))
    segments.append( Node(Segment(148, '4127', 'MEM', 'HRO', 500, 585, 1, 1,HomeBases)))
    segments.append( Node(Segment(153, '4128', 'HRO', 'MEM', 615, 695, 1, 1,HomeBases)))
    segments.append( Node(Segment(158, '4311', 'HRO', 'MEM', 390, 470, 1, 1,HomeBases)))
    segments.append( Node(Segment(164, '4311', 'HRO', 'MEM', 390, 470, 1, 1,HomeBases)))
    segments.append( Node(Segment(169, '4311', 'HRO', 'MEM', 390, 470, 1, 1,HomeBases)))
    segments.append( Node(Segment(175, '4316', 'MEM', 'HRO', 1135, 1220, 1, 1,HomeBases)))
    return(segments)

def buildSet3():
    segments=[]
    segments.append( Node(Segment(73, '3312', 'PDX', 'PDT', 420, 500, 1, 1,HomeBases)))
    segments.append( Node(Segment(78, '3314', 'PDX', 'PDT', 635, 715, 1, 1,HomeBases)))
    segments.append( Node(Segment(83, '3316', 'PDX', 'PDT', 920, 1000, 1, 1,HomeBases)))
    segments.append( Node(Segment(88, '3318', 'PDX', 'PDT', 1135, 1215, 1, 1,HomeBases)))
    segments.append( Node(Segment(97, '3318', 'PDX', 'PDT', 1135, 1215, 1, 1,HomeBases)))
    segments.append( Node(Segment(103, '3321', 'PDT', 'PDX', 315, 395, 1, 1,HomeBases)))
    segments.append( Node(Segment(108, '3323', 'PDT', 'PDX', 525, 605, 1, 1,HomeBases)))
    segments.append( Node(Segment(113, '3325', 'PDT', 'PDX', 740, 820, 1, 1,HomeBases)))
    segments.append( Node(Segment(118, '3327', 'PDT', 'PDX', 1025, 1105, 1, 1,HomeBases)))
    segments.append( Node(Segment(126, '3327', 'PDT', 'PDX', 1025, 1105, 1, 1,HomeBases)))
    return(segments)


In [ ]:


#segments = buildSet1()
G1 = gen.buildViewGraph(segments)
#G2 = buildViewGraph(buildSet2())
#G3 = buildViewGraph(buildSet3())

pos = nx.circular_layout(G1)
nx.draw_networkx(G1, pos=pos, node_size=400 , alpha=1)

#pos = nx.kamada_kawai_layout(G2)
#nx.draw_networkx(G2, pos=pos, node_size=400 , alpha=1)

#pos = nx.circular_layout(G3)
#nx.draw_networkx(G3, pos=pos, node_size=400 , alpha=1)

#plot.graph(G2) #Example with G2


In [ ]:
G = gen.getG() # gen.buildFltGraph(segments)

# Init the number of segment Nodes

N = gen.getN() # len(segments)
print( "N is ", N)

# T is the time range encompassing all segments 

T = gen.getT() # max(node.obj.getUarrtime() for node in segments) + (2 * 1440) # We add buffer of 1 day prior and 1 day after
print ( "T is ", T)




# For each contraint active, adjust the offset to show a 0 energy for respected constraints
BQM_offset = 0


for n1,n2,data in gen.getG().edges(data=True):
    print("Seg %d connects to %d with a time gap %d minutes. Base switch is %d" % (n1.obj.id,n2.obj.id, data["weight"].gap, data["weight"].TransitionBaseWeight))



In [ ]:


print(sampleset.first.sample.keys() )
print(sampleset.first.energy)
print(sampleset)




In [ ]:

tg.print_all(sampleset)
